# Multi-Stream Neural Networks: CIFAR100 Training

This notebook demonstrates training Multi-Stream Neural Networks (MSNN) on the CIFAR100 dataset.

The notebook follows this structure:
1. Environment setup and repository update
2. Installation of dependencies and importing libraries
3. Data loading, processing, verification, visualization and analysis 
4. Model creation, training and evaluation
5. Pathway analysis and model saving

## Multi-Stream Neural Networks

This notebook demonstrates the full pipeline for training multi-stream neural networks:

### Key Features
- **Unified API Design**: Consistent interface across all models
- **Two Fusion Strategies**: Shared classifier (recommended) vs separate classifiers
- **Multiple Architectures**: Dense networks and CNN (ResNet) models
- **GPU Optimization**: Automatic device detection with mixed precision
- **Research Tools**: Pathway analysis for multi-stream insights

### Model Architectures
1. **BaseMultiChannelNetwork**: Dense/fully-connected multi-stream processing
2. **MultiChannelResNetNetwork**: CNN with residual connections for spatial features

### API Design Philosophy
- **`model(color, brightness)`** → Single tensor for training/inference
- **`model.analyze_pathways(color, brightness)`** → Tuple for research analysis
- **Keras-like training**: `.fit()`, `.evaluate()`, `.predict()` methods


## Environment Setup & Requirements

### Prerequisites
- **Python 3.8+**
- **PyTorch 1.12+** with CUDA support (recommended)
- **Google Colab** (this notebook) or local Jupyter environment

### Project Structure
Our codebase is fully modularized:
```
Multi-Stream-Neural-Networks/
├── src/
│   ├── models/basic_multi_channel/     # Core model implementations
│   │   ├── base_multi_channel_network.py    # Dense model
│   │   └── multi_channel_resnet_network.py  # CNN model
│   ├── utils/cifar100_loader.py        # CIFAR-100 data utilities
│   ├── transforms/rgb_to_rgbl.py       # RGB→Brightness transform
│   └── utils/device_utils.py           # GPU optimization utilities
├── configs/                            # Model configuration files
└── data/                               # Dataset location
```


## 1. Environment Setup

Mount Google Drive and navigate to the project directory.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Navigate to the project directory
import os

# Navigate to Drive and project directory
project_path = '/content/drive/MyDrive/Multi-Stream-Neural-Networks'
if os.path.exists(project_path):
    os.chdir(project_path)
    print(f"✅ Found project at: {project_path}")
else:
    print(f"❌ Project not found at: {project_path}")
    print("💡 Please clone the repository first:")
    print("   !git clone https://github.com/yourusername/Multi-Stream-Neural-Networks.git")

## 2. Update Repository

Pull the latest changes from the repository to ensure we have the most recent codebase.


In [ ]:
# Update repository with latest changes
print("🔄 Pulling latest changes from repository...")

# Make sure we're in the right directory
print(f"📁 Current directory: {os.getcwd()}")

# Pull latest changes
!git pull origin main

print("\n✅ Repository update complete!")

## 3. Install Dependencies

Install required packages and dependencies for the project.

In [ ]:
# Install Dependencies
print("📦 Installing required dependencies...")

import subprocess
import sys
import importlib

def install_package(package):
    """Install a package if not already installed."""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--quiet"])
        return True
    except subprocess.CalledProcessError:
        return False

# Required packages
packages = [
    "torch",
    "torchvision", 
    "numpy",
    "matplotlib",
    "seaborn",
    "tqdm",
    "scikit-learn",
    "Pillow"
]

print("Installing packages...")
for package in packages:
    if install_package(package):
        print(f"✅ {package}")
    else:
        print(f"❌ Failed to install {package}")

# Install project requirements
print("\nInstalling project requirements...")
try:
    !pip install -r requirements.txt
    print("✅ Project requirements installed")
except Exception as e:
    print(f"⚠️ Error installing requirements: {e}")
    print("   Continuing with available packages")


📦 Installing required dependencies...
Installing packages...



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


✅ torch



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


✅ torchvision


...


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


✅ tqdm



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


✅ scikit-learn
✅ Pillow

📚 Importing libraries...
✅ All libraries imported successfully!

🔧 PyTorch Setup:
   PyTorch version: 2.7.0
   CUDA available: False
   Using CPU (consider GPU for faster training)

🎯 Dependencies and imports complete!
✅ Pillow

📚 Importing libraries...
✅ All libraries imported successfully!

🔧 PyTorch Setup:
   PyTorch version: 2.7.0
   CUDA available: False
   Using CPU (consider GPU for faster training)

🎯 Dependencies and imports complete!



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


## 4. Import Libraries

Import all necessary libraries and utilities for the project.

In [ ]:
# Import Libraries
print("📚 Importing libraries and setting up the environment...")

#------------------------------------------------------------------------------
# Core PyTorch Libraries
#------------------------------------------------------------------------------
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from torchvision.models import resnet50, ResNet50_Weights

#------------------------------------------------------------------------------
# Data Handling Libraries
#------------------------------------------------------------------------------
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import os
import sys
import json
import time
import subprocess
import importlib
import traceback

#------------------------------------------------------------------------------
# Visualization Libraries
#------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

#------------------------------------------------------------------------------
# Progress Tracking and Machine Learning Libraries
#------------------------------------------------------------------------------
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

#------------------------------------------------------------------------------
# Google Colab Integration (if needed)
#------------------------------------------------------------------------------
try:
    from google.colab import drive
    IN_COLAB = True
    print("✅ Google Colab detected")
except ImportError:
    IN_COLAB = False
    print("ℹ️ Not running in Google Colab")

#------------------------------------------------------------------------------
# Project-Specific Imports
#------------------------------------------------------------------------------
# Add project root to path for imports
project_root = Path('.').resolve()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Import project modules (with error handling)
try:
    # Data processing
    from src.utils.cifar100_loader import get_cifar100_datasets, create_validation_split
    from src.transforms.rgb_to_rgbl import RGBtoRGBL
    
    # Data augmentation
    from src.transforms.augmentation import (
        CIFAR100Augmentation, 
        AugmentedMultiStreamDataset,
        MixUp, 
        create_augmented_dataloaders,
        create_test_dataloader
    )
    
    # Model builders
    try:
        from src.models.builders import create_model, list_available_models
        MODEL_FACTORY_AVAILABLE = True
    except ImportError:
        MODEL_FACTORY_AVAILABLE = False
        from src.models.basic_multi_channel.base_multi_channel_network import BaseMultiChannelNetwork, base_multi_channel_large
        from src.models.basic_multi_channel.multi_channel_resnet_network import MultiChannelResNetNetwork, multi_channel_resnet50
    
    print("✅ Project modules imported successfully")
except ImportError as e:
    print(f"⚠️ Error importing project modules: {e}")
    print("   Some functionality may be limited")

# Set environment variables for CUDA
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'  # Better error diagnostics for CUDA

print("✅ All libraries imported successfully!")

#------------------------------------------------------------------------------
# Environment Information
#------------------------------------------------------------------------------
# Check PyTorch setup
print(f"\n🔧 PyTorch Setup:")
print(f"   PyTorch version: {torch.__version__}")
print(f"   CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"   CUDA version: {torch.version.cuda}")
else:
    print("   Using CPU (consider GPU for faster training)")

# Configure CUDA for better performance
torch.backends.cudnn.benchmark = True  # Speed up training for fixed input sizes
torch.backends.cudnn.deterministic = False  # Non-deterministic for speed

print("\n🎯 Environment setup complete!")

## 5. Load Data

Load the CIFAR-100 dataset using our optimized data loader.

In [ ]:
# 📊 CIFAR-100 Data Loading and Verification
print("📁 Setting up CIFAR-100 dataset loading...")

# Load CIFAR-100 Dataset
print("📁 Loading CIFAR-100 datasets with train/validation/test split...")

# Since we already imported the CIFAR-100 loader utilities in the main import cell,
# we can check if they were imported successfully
if 'get_cifar100_datasets' in globals():
    print("✅ CIFAR-100 loader utilities imported successfully")
else:
    print("❌ Failed to import CIFAR-100 utilities. Make sure src/utils/cifar100_loader.py exists")
    raise ImportError("Missing CIFAR-100 loader utilities")

# Load datasets using our optimized loader (returns train, test, class_names)
train_dataset, test_dataset, class_names = get_cifar100_datasets(
    data_dir='./data/cifar-100'
)

# Create validation split from training data
train_dataset, val_dataset = create_validation_split(
    train_dataset, 
    val_split=0.1
)

print("✅ CIFAR-100 datasets loaded successfully!")
print(f"   📊 Training samples: {len(train_dataset):,}")
print(f"   📊 Validation samples: {len(val_dataset):,}")
print(f"   📊 Test samples: {len(test_dataset):,}")
print(f"   🏷️ Number of classes: {len(class_names)}")

# Store class names for later use
CIFAR100_FINE_LABELS = class_names

print("\n🎯 Data loading complete!")

📁 Setting up CIFAR-100 dataset loading...
✅ CIFAR-100 loader utilities imported successfully
📁 Loading CIFAR-100 datasets with train/validation/test split...
❌ Error loading CIFAR-100 data: get_cifar100_datasets() got an unexpected keyword argument 'root'

💡 Troubleshooting:
   1. Check internet connection for CIFAR-100 download
   2. Verify data directory permissions
   3. Try clearing cache: rm -rf data/cifar-100
   4. Check if src/utils/cifar100_loader.py exists


TypeError: get_cifar100_datasets() got an unexpected keyword argument 'root'

## 6. Process Data

Convert RGB images to RGB + Brightness (L) channels for multi-stream processing.

In [ ]:
# Process Data: RGB to RGB+L (Brightness) Conversion
print("🔄 Converting RGB images to RGB + Brightness streams...")

# Since we already imported the RGB to RGB+L transform in the main import cell,
# we can check if it was imported successfully
if 'RGBtoRGBL' in globals():
    print("✅ RGB to RGB+L transform imported successfully")
else:
    print("❌ Failed to import RGB to RGB+L transform. Make sure src/transforms/rgb_to_rgbl.py exists")
    raise ImportError("Missing RGB to RGB+L transform")

# Initialize the transform
rgb_to_rgbl = RGBtoRGBL()

# Function to process a dataset batch-wise for memory efficiency
def process_dataset_to_streams(dataset, batch_size=1000, desc="Processing"):
    """
    Convert RGB dataset to RGB + Brightness streams efficiently.
    
    Args:
        dataset: Dataset with RGB images (PyTorch dataset format)
        batch_size: Size of batches for memory-efficient processing
        desc: Description for progress bar
        
    Returns:
        Tuple of (rgb_stream, brightness_stream, labels_tensor)
    """
    rgb_tensors = []
    brightness_tensors = []
    labels = []
    
    # Process in batches to manage memory
    for i in tqdm(range(0, len(dataset), batch_size), desc=desc):
        batch_end = min(i + batch_size, len(dataset))
        batch_data = []
        batch_labels = []
        
        # Collect batch data
        for j in range(i, batch_end):
            data, label = dataset[j]
            batch_data.append(data)
            batch_labels.append(label)
        
        # Convert to tensor batch
        batch_tensor = torch.stack(batch_data)
        
        # Apply RGB to RGB+L transform
        rgb_batch, brightness_batch = rgb_to_rgbl(batch_tensor)
        
        rgb_tensors.append(rgb_batch)
        brightness_tensors.append(brightness_batch)
        labels.extend(batch_labels)
    
    # Concatenate all batches
    rgb_stream = torch.cat(rgb_tensors, dim=0)
    brightness_stream = torch.cat(brightness_tensors, dim=0)
    labels_tensor = torch.tensor(labels, dtype=torch.long)
    
    return rgb_stream, brightness_stream, labels_tensor

# Process all datasets
print("Processing training dataset...")
train_rgb, train_brightness, train_labels_tensor = process_dataset_to_streams(
    train_dataset, desc="Training data"
)

print("Processing validation dataset...")
val_rgb, val_brightness, val_labels_tensor = process_dataset_to_streams(
    val_dataset, desc="Validation data"
)

print("Processing test dataset...")
test_rgb, test_brightness, test_labels_tensor = process_dataset_to_streams(
    test_dataset, desc="Test data"
)

print("\n✅ Multi-stream conversion complete!")
print(f"   🎨 RGB stream shape: {train_rgb.shape}")
print(f"   💡 Brightness stream shape: {train_brightness.shape}")
print(f"   📊 RGB range: [{train_rgb.min():.3f}, {train_rgb.max():.3f}]")
print(f"   📊 Brightness range: [{train_brightness.min():.3f}, {train_brightness.max():.3f}]")

# Memory usage estimation
rgb_memory = (train_rgb.nbytes + val_rgb.nbytes + test_rgb.nbytes) / 1e6
brightness_memory = (train_brightness.nbytes + val_brightness.nbytes + test_brightness.nbytes) / 1e6
total_memory = rgb_memory + brightness_memory

print(f"\n📈 Processing Summary:")
print(f"   📊 Total samples processed: {len(train_labels_tensor) + len(val_labels_tensor) + len(test_labels_tensor):,}")
print(f"   🎨 RGB streams memory: {rgb_memory:.1f} MB")
print(f"   💡 Brightness streams memory: {brightness_memory:.1f} MB")
print(f"   💾 Total memory usage: {total_memory:.1f} MB")

print("\n🎯 Data processing complete!")

## 7. Data Verification

Verify the processed data structure and consistency.

In [ ]:
# Data Verification
print("🔍 Verifying processed data structure and consistency...")

def verify_data_integrity(rgb_data, brightness_data, labels, split_name):
    # Check shapes and types
    assert rgb_data.shape[0] == brightness_data.shape[0] == labels.shape[0], f"Inconsistent sample counts in {split_name}!"
    assert rgb_data.shape[1:] == (3, 32, 32), f"Unexpected RGB shape in {split_name}!"
    assert brightness_data.shape[1:] == (1, 32, 32), f"Unexpected brightness shape in {split_name}!"
    assert 0 <= labels.min() and labels.max() < 100, f"Invalid label range in {split_name}!"
    return rgb_data.shape[0]

train_samples = verify_data_integrity(train_rgb, train_brightness, train_labels_tensor, "Training")
val_samples = verify_data_integrity(val_rgb, val_brightness, val_labels_tensor, "Validation")
test_samples = verify_data_integrity(test_rgb, test_brightness, test_labels_tensor, "Test")

total_samples = train_samples + val_samples + test_samples
all_labels = torch.cat([train_labels_tensor, val_labels_tensor, test_labels_tensor])
unique_labels = torch.unique(all_labels)

print(f"\n📈 Data Summary:")
print(f"   Training: {train_samples:,}")
print(f"   Validation: {val_samples:,}")
print(f"   Test: {test_samples:,}")
print(f"   Total: {total_samples:,}")
print(f"   Unique classes: {len(unique_labels)}/100")
print("\n✅ Data verification checks passed!")

## 8. Data Visualization

Visualize sample images from both RGB and brightness streams.

In [ ]:
# Data Visualization
print("👁️ Visualizing sample images from both RGB and brightness streams...")

# Set up visualization
plt.style.use('default')
fig, axes = plt.subplots(2, 4, figsize=(12, 6))
fig.suptitle('Multi-Stream CIFAR-100 Samples: RGB vs Brightness', fontsize=14)

# Select random samples
np.random.seed(42)  # For reproducible results
sample_indices = np.random.choice(len(train_rgb), 4, replace=False)

for i, idx in enumerate(sample_indices):
    # Get data
    rgb_img = train_rgb[idx]
    brightness_img = train_brightness[idx]
    label = train_labels_tensor[idx].item()
    class_name = CIFAR100_FINE_LABELS[label]
    
    # RGB image (convert from tensor to numpy)
    rgb_np = rgb_img.permute(1, 2, 0).numpy()
    rgb_np = np.clip(rgb_np, 0, 1)  # Ensure valid range
    
    # Brightness image
    brightness_np = brightness_img.squeeze().numpy()
    
    # Plot RGB
    axes[0, i].imshow(rgb_np)
    axes[0, i].set_title(f'RGB: {class_name}', fontsize=10)
    axes[0, i].axis('off')
    
    # Plot Brightness
    axes[1, i].imshow(brightness_np, cmap='gray')
    axes[1, i].set_title(f'Brightness: {class_name}', fontsize=10)
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

print("✅ Data visualization complete!")

## 9. Data Analysis

Perform basic data analysis on class distribution and stream characteristics.

In [ ]:
# Data Analysis
print("📊 Performing basic data analysis...")

# Set up matplotlib for better visualizations
plt.style.use('default')
sns.set_palette("husl")

# Class Distribution Analysis
print("\n🏷️ Analyzing class distribution...")

# Training distribution
train_counts = np.bincount(train_labels_tensor.numpy(), minlength=100)
plt.figure(figsize=(10, 4))
plt.bar(range(100), train_counts, alpha=0.7, color='skyblue')
plt.title('Training Set Class Distribution', fontweight='bold')
plt.xlabel('Class ID')
plt.ylabel('Sample Count')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Stream Statistics Analysis
print("\n🎨 RGB vs Brightness stream characteristics:")

# Sample a subset for analysis
sample_size = min(1000, len(train_rgb))
indices = np.random.choice(len(train_rgb), sample_size, replace=False)
rgb_sample = train_rgb[indices]
brightness_sample = train_brightness[indices]

# Calculate statistics
rgb_stats = {
    'mean': rgb_sample.mean().item(),
    'std': rgb_sample.std().item(),
    'min': rgb_sample.min().item(),
    'max': rgb_sample.max().item()
}

brightness_stats = {
    'mean': brightness_sample.mean().item(),
    'std': brightness_sample.std().item(), 
    'min': brightness_sample.min().item(),
    'max': brightness_sample.max().item()
}

print(f"   🎨 RGB statistics:")
print(f"      Mean: {rgb_stats['mean']:.3f}, Std: {rgb_stats['std']:.3f}")
print(f"      Min: {rgb_stats['min']:.3f}, Max: {rgb_stats['max']:.3f}")
print(f"   💡 Brightness statistics:")
print(f"      Mean: {brightness_stats['mean']:.3f}, Std: {brightness_stats['std']:.3f}")
print(f"      Min: {brightness_stats['min']:.3f}, Max: {brightness_stats['max']:.3f}")

print("\n✅ Data analysis complete!")

## 10. Data Augmentation

Set up data augmentation for multi-stream training using the project's CIFAR-100 augmentation module.

Data augmentation helps improve model generalization by creating variations of the training data. In this notebook, we'll use on-the-fly augmentation through PyTorch DataLoaders, which:

1. Applies transformations randomly during training
2. Creates unique augmentations in each epoch
3. Preserves the original data while generating variations

We'll create augmented DataLoaders that:
- Apply transformations to the original data during batch loading
- Handle both RGB and brightness streams consistently
- Can be used directly with our models through the `fit_dataloader()` method

This approach is consistent with the standard PyTorch best practices for data augmentation.

In [ ]:
# Data Augmentation
print("🔄 Setting up data augmentation using project's implementation...")

# Create augmentation with custom settings for CIFAR-100
augmentation_config = {
    'horizontal_flip_prob': 0.5,  # 50% chance of flipping horizontally
    'rotation_degrees': 10.0,     # Rotate up to ±10 degrees
    'translate_range': 0.1,       # Translate up to 10% of image size
    'scale_range': (0.9, 1.1),    # Scale between 90-110%
    'color_jitter_strength': 0.3, # Moderate color jittering
    'gaussian_noise_std': 0.01,   # Small amount of noise
    'cutout_prob': 0.3,           # 30% chance of applying cutout
    'cutout_size': 8,             # 8x8 pixel cutout
    'enabled': True               # Enable augmentation
}

# Setup MixUp augmentation
mixup_alpha = 0.2  # Alpha parameter for Beta distribution

# Create augmented datasets and data loaders in one step
print("\n📊 Creating augmented DataLoaders...")
train_loader, val_loader = create_augmented_dataloaders(
    train_rgb, train_brightness, train_labels_tensor,  # Training data
    val_rgb, val_brightness, val_labels_tensor,        # Validation data
    batch_size=512,                                    # Increased batch size for A100 GPU
    dataset="cifar100",                                # Dataset type
    augmentation_config=augmentation_config,           # Augmentation settings
    mixup_alpha=mixup_alpha,                           # MixUp parameter
    num_workers=2,                                     # Parallel workers
    pin_memory=torch.cuda.is_available()               # Pin memory if GPU available
)

# Create test dataloader separately (no augmentation)
test_loader = create_test_dataloader(
    test_rgb, test_brightness, test_labels_tensor,
    batch_size=512,                                    # Increased batch size for A100 GPU
    num_workers=2,
    pin_memory=torch.cuda.is_available()
)

print("\n✅ Data augmentation setup complete")
print("   Using project's CIFAR-100 specific augmentations:")
print(f"   - Horizontal flips: {augmentation_config['horizontal_flip_prob']}")
print(f"   - Rotation: ±{augmentation_config['rotation_degrees']}°")
print(f"   - Translation: ±{augmentation_config['translate_range'] * 100}%")
print(f"   - Color jitter strength: {augmentation_config['color_jitter_strength']}")
print(f"   - Gaussian noise (std): {augmentation_config['gaussian_noise_std']}")
print(f"   - Cutout: {augmentation_config['cutout_prob']} probability, {augmentation_config['cutout_size']}px")
print(f"   - MixUp alpha: {mixup_alpha}")
print(f"\n   Training batches: {len(train_loader)}")
print(f"   Validation batches: {len(val_loader)}")
print(f"   Test batches: {len(test_loader)}")

## 11. Prepare Data for Training

Create DataLoaders with the processed data for model training.

In [ ]:
# Prepare Data for Training
print("🔄 Preparing data for model training...")

# The data loaders were already created in the previous cell:
#  - train_loader: Training data with augmentation
#  - val_loader: Validation data without augmentation
#  - test_loader: Test data without augmentation

# Confirm data loader settings
print(f"\n📦 DataLoader configuration:")
print(f"   Batch size: {next(iter(train_loader))[0].shape[0]}")
print(f"   Number of workers: 2")
print(f"   Pin memory: {torch.cuda.is_available()}")
print(f"   Training with augmentation: Yes")
print(f"   Training with MixUp: {'Yes' if mixup_alpha is not None else 'No'}")

# Now let's prepare the data in both formats needed for different model types:
# 1. Original format (N, C, H, W) for CNN models
# 2. Reshaped format (N, C*H*W) for dense models

print("\n🔄 Preparing data in both formats for different model architectures...")

# For CNN models: keep the original tensor format (N, C, H, W)
# We already have these as train_rgb, train_brightness, etc.
train_color_data = train_rgb.cpu().numpy()  # Original shape (N, C, H, W)
train_brightness_data = train_brightness.cpu().numpy()  # Original shape (N, C, H, W)
val_color_data = val_rgb.cpu().numpy()  # Original shape (N, C, H, W)
val_brightness_data = val_brightness.cpu().numpy()  # Original shape (N, C, H, W)
test_color_data = test_rgb.cpu().numpy()  # Original shape (N, C, H, W)
test_brightness_data = test_brightness.cpu().numpy()  # Original shape (N, C, H, W)

# Convert labels to numpy arrays
train_labels = train_labels_tensor.cpu().numpy()
val_labels = val_labels_tensor.cpu().numpy()
test_labels = test_labels_tensor.cpu().numpy()

print(f"CNN data shapes:")
print(f"   RGB train: {train_color_data.shape}, Brightness train: {train_brightness_data.shape}")
print(f"   RGB val: {val_color_data.shape}, Brightness val: {val_brightness_data.shape}")
print(f"   RGB test: {test_color_data.shape}, Brightness test: {test_brightness_data.shape}")

# Prepare flattened data for dense models (flattening the spatial dimensions)
# Reshape from (N, C, H, W) to (N, C*H*W)
train_color_data_reshaped = train_color_data.reshape(train_color_data.shape[0], -1)
train_brightness_data_reshaped = train_brightness_data.reshape(train_brightness_data.shape[0], -1)
val_color_data_reshaped = val_color_data.reshape(val_color_data.shape[0], -1)
val_brightness_data_reshaped = val_brightness_data.reshape(val_brightness_data.shape[0], -1)
test_color_data_reshaped = test_color_data.reshape(test_color_data.shape[0], -1)
test_brightness_data_reshaped = test_brightness_data.reshape(test_brightness_data.shape[0], -1)

print(f"\nDense data shapes (flattened):")
print(f"   RGB train: {train_color_data_reshaped.shape}, Brightness train: {train_brightness_data_reshaped.shape}")
print(f"   RGB val: {val_color_data_reshaped.shape}, Brightness val: {val_brightness_data_reshaped.shape}")
print(f"   RGB test: {test_color_data_reshaped.shape}, Brightness test: {test_brightness_data_reshaped.shape}")

print("\n✅ Data preparation complete for both CNN and dense models!")

# For dense models: also create these variables for compatibility with existing code
train_color_data_reshaped = train_color_data.reshape(train_color_data.shape[0], -1)
train_brightness_data_reshaped = train_brightness_data.reshape(train_brightness_data.shape[0], -1)
val_color_data_reshaped = val_color_data.reshape(val_color_data.shape[0], -1)
val_brightness_data_reshaped = val_brightness_data.reshape(val_brightness_data.shape[0], -1)
test_color_data_reshaped = test_color_data.reshape(test_color_data.shape[0], -1)
test_brightness_data_reshaped = test_brightness_data.reshape(test_brightness_data.shape[0], -1)

# Calculate expected dimensions
rgb_input_size = 3 * 32 * 32  # 3 channels x 32 x 32 for CIFAR-100
brightness_input_size = 1 * 32 * 32  # 1 channel x 32 x 32 for CIFAR-100

print(f"✅ Dense format ready - RGB: {train_color_data_reshaped.shape}, Brightness: {train_brightness_data_reshaped.shape}")

# Verify dimensions
if train_color_data_reshaped.shape[1] != rgb_input_size:
    print(f"⚠️ Warning: Reshaped RGB size {train_color_data_reshaped.shape[1]} doesn't match expected size {rgb_input_size}")
if train_brightness_data_reshaped.shape[1] != brightness_input_size:
    print(f"⚠️ Warning: Reshaped brightness size {train_brightness_data_reshaped.shape[1]} doesn't match expected size {brightness_input_size}")

# Make sure CUDA_LAUNCH_BLOCKING is set for more detailed error messages
if 'CUDA_LAUNCH_BLOCKING' not in os.environ:
    os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Configure CUDA for better performance
torch.backends.cudnn.benchmark = True  # Speed up training for fixed input sizes
torch.backends.cudnn.deterministic = False  # Non-deterministic for speed

# Sample batch for verification
sample_color, sample_brightness, sample_labels = next(iter(train_loader))
print(f"\n📊 Sample batch shapes from data loader:")
print(f"   Color batch: {sample_color.shape}")
print(f"   Brightness batch: {sample_brightness.shape}")
print(f"   Labels batch: {sample_labels.shape}")

print("\n🎯 All data formats are ready for model training!")

# Initialize some training parameters that will be used by multiple models
if 'num_epochs' not in globals():
    num_epochs = 20
if 'early_stopping_patience' not in globals():
    early_stopping_patience = 5

print(f"\n🔧 Training Configuration:")
print(f"   Default epochs: {num_epochs}")
print(f"   Default early stopping patience: {early_stopping_patience}")
print(f"   Default batch size: 512 (optimized for A100 GPU)")

## 12. Create and train Baseline ResNet50 Model

Create a standard ResNet50 model for comparison with multi-stream models.

In [ ]:
# Create Baseline ResNet50 Model
print("🏗️ Creating baseline ResNet50 model for comparison...")

# Check GPU availability and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ Using device: {device}")

# Create ResNet50 model modified for CIFAR-100
class CifarResNet50(nn.Module):
    def __init__(self, num_classes=100, pretrained=False):
        super(CifarResNet50, self).__init__()
        
        # Load ResNet50 with or without pretrained weights
        if pretrained:
            print("   Using pretrained weights (ImageNet)")
            self.model = resnet50(weights=ResNet50_Weights.DEFAULT)
        else:
            print("   Using randomly initialized weights for fair comparison")
            self.model = resnet50(weights=None)
        
        # Modify first conv layer to work with 32x32 CIFAR images instead of 224x224 ImageNet
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        # Remove maxpool to preserve spatial dimensions for small images
        self.model.maxpool = nn.Identity()
        
        # Replace final fully connected layer for CIFAR-100
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
        
        # Move to device
        self.to(device)
    
    def forward(self, x):
        return self.model(x)

# Create the model - explicitly using pretrained=False for fair comparison with multi-stream models
print("\n⚠️ IMPORTANT: Using NON-PRETRAINED weights for fair comparison with multi-stream models")
baseline_model = CifarResNet50(num_classes=100, pretrained=False)
baseline_model = baseline_model.to(device)

# Setup optimizer and loss with weight decay for regularization
baseline_optimizer = optim.Adam(baseline_model.parameters(), lr=0.001, weight_decay=0.0001)
baseline_criterion = nn.CrossEntropyLoss()

# Count parameters
baseline_params = sum(p.numel() for p in baseline_model.parameters())
baseline_trainable = sum(p.numel() for p in baseline_model.parameters() if p.requires_grad)

print(f"\n✅ Baseline ResNet50 created successfully")
print(f"   Architecture: Modified ResNet50 for CIFAR-100")
print(f"   Total parameters: {baseline_params:,}")
print(f"   Trainable parameters: {baseline_trainable:,}")
print(f"   Input shape: (3, 32, 32)")
print(f"   Device: {device}")
print(f"   Initialization: Random (not pretrained)")
print(f"   Optimizer: Adam with weight_decay=0.0001")

# Quick validation on dummy data
print("\n🏋️‍♂️ Validating model with dummy data...")
dummy_data = torch.randn(8, 3, 32, 32, device=device)
dummy_labels = torch.randint(0, 100, (8,), device=device)

baseline_model.train()
outputs = baseline_model(dummy_data)
loss = baseline_criterion(outputs, dummy_labels)
print(f"   ✅ Forward pass successful, loss: {loss.item():.4f}")

# Reset gradients
baseline_optimizer.zero_grad()
loss.backward()
baseline_optimizer.step()
print(f"   ✅ Backward pass successful")

print("\n✅ Baseline model ready for training!")

# Immediate training on dummy data for validation
print("🏋️‍♂️ Starting immediate training on dummy data...")

# Create dummy CIFAR-100 data
dummy_data = torch.randn(64, 3, 32, 32, device=device)  # 64 samples, 3 channels, 32x32 images
dummy_labels = torch.randint(0, 100, (64,), device=device)  # 64 labels for 100 classes

# Training loop (1 epoch)
baseline_model.train()
for epoch in range(1):  # Just 1 epoch for quick testing
    # Forward pass
    outputs = baseline_model(dummy_data)
    loss = baseline_criterion(outputs, dummy_labels)
    
    # Backward pass and optimization
    baseline_optimizer.zero_grad()
    loss.backward()
    baseline_optimizer.step()
    
    print(f"Epoch [{epoch+1}/1], Loss: {loss.item():.4f}")

print("✅ Immediate training completed!")

In [ ]:
# Train Baseline ResNet50 Model
print("🏋️‍♀️ Training baseline ResNet50 model...")

# Define training function with early stopping and learning rate scheduling
def train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs=100, patience=10, model_name="Model"):
    """
    Train a model and return training history.
    
    Args:
        model: The model to train
        train_loader: DataLoader for training data
        val_loader: DataLoader for validation data
        optimizer: Optimizer to use
        criterion: Loss function
        num_epochs: Number of epochs to train
        patience: Early stopping patience
        model_name: Name for logging
        
    Returns:
        Dictionary with training history
    """
    device = next(model.parameters()).device
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    
    best_val_acc = 0.0
    best_model_state = None
    no_improvement_count = 0
    
    # Learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 
        mode='max',
        factor=0.5,
        patience=5,
        verbose=True
    )
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        train_bar = tqdm(train_loader, desc=f"{model_name} Training")
        
        for batch_idx, data in enumerate(train_bar):
            # Handle the baseline model which only needs RGB data - support both tuple and list types
            if isinstance(data, (tuple, list)) and len(data) == 3:
                rgb, _, targets = data  # Unpack RGB, ignore brightness
            else:
                # Fallback for unexpected data format
                rgb, targets = data
                
            # Move to device
            rgb, targets = rgb.to(device), targets.to(device)
            
            # Zero gradients
            optimizer.zero_grad()
            
            # Forward pass - baseline model only takes RGB
            outputs = model(rgb)
            loss = criterion(outputs, targets)
            
            # Add L2 regularization term (additional to weight decay in optimizer)
            l2_lambda = 0.0001
            l2_reg = 0.0
            for param in model.parameters():
                l2_reg += torch.norm(param, 2)
            loss += l2_lambda * l2_reg
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            # Track statistics
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += targets.size(0)
            train_correct += predicted.eq(targets).sum().item()
            
            # Update progress bar
            train_bar.set_postfix({
                'loss': train_loss/(batch_idx+1), 
                'acc': 100.*train_correct/train_total
            })
        
        train_acc = 100. * train_correct / train_total
        train_loss = train_loss / len(train_loader)
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            val_bar = tqdm(val_loader, desc=f"{model_name} Validation")
            
            for batch_idx, data in enumerate(val_bar):
                # Handle data format - support both tuple and list types
                if isinstance(data, (tuple, list)) and len(data) == 3:
                    rgb, _, targets = data  # Unpack RGB, ignore brightness
                else:
                    rgb, targets = data
                
                # Move to device
                rgb, targets = rgb.to(device), targets.to(device)
                
                # Forward pass
                outputs = model(rgb)
                loss = criterion(outputs, targets)
                
                # Track statistics
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += targets.size(0)
                val_correct += predicted.eq(targets).sum().item()
                
                # Update progress bar
                val_bar.set_postfix({
                    'loss': val_loss/(batch_idx+1), 
                    'acc': 100.*val_correct/val_total
                })
        
        val_acc = 100. * val_correct / val_total
        val_loss = val_loss / len(val_loader)
        
        # Save history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        # Print epoch summary
        print(f"Epoch {epoch+1}/{num_epochs} Summary:")
        print(f"   Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"   Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        print(f"   Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
        
        # Update learning rate scheduler
        scheduler.step(val_acc)
        
        # Track best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict().copy()
            no_improvement_count = 0
            print(f"   ✅ New best validation accuracy: {best_val_acc:.2f}%")
        else:
            no_improvement_count += 1
            print(f"   ⚠️ No improvement for {no_improvement_count} epochs")
        
        # Early stopping check
        if no_improvement_count >= patience:
            print(f"   🛑 Early stopping triggered after {epoch+1} epochs")
            break
    
    # Restore best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    print(f"\n✅ {model_name} training complete!")
    print(f"   Best validation accuracy: {best_val_acc:.2f}%")
    
    return history

# Training configuration
num_epochs = 100  # Maximum epochs
patience = 10     # Early stopping patience
batch_size = 512  # Increased batch size for A100 GPU

print(f"\n🔧 Training Configuration:")
print(f"   Epochs: {num_epochs} (with early stopping, patience={patience})")
print(f"   Batch size: {batch_size} (optimized for A100 GPU)")
print(f"   Optimizer: Adam with weight decay=0.0001")
print(f"   Learning rate: 0.001 with ReduceLROnPlateau scheduling")
print(f"   Regularization: L2 weight decay")
print(f"   Early stopping: Patience {patience} epochs")
print(f"   Loss: CrossEntropyLoss")
print(f"   GPU acceleration: {torch.cuda.is_available()}")
print(f"   GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

# Train baseline model
baseline_history = train_model(
    model=baseline_model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=baseline_optimizer,
    criterion=baseline_criterion,
    num_epochs=num_epochs,
    patience=patience,
    model_name="Baseline ResNet50"
)

# Visualize training curves
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(baseline_history['train_acc'], label='Train')
plt.plot(baseline_history['val_acc'], label='Validation')
plt.title('Baseline ResNet50 Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(baseline_history['train_loss'], label='Train')
plt.plot(baseline_history['val_loss'], label='Validation')
plt.title('Baseline ResNet50 Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Baseline model training complete!")

## 13. Create Multi-stream models

Now that we've trained our baseline ResNet50 model as a reference, let's create and train our multi-stream neural network models.

The multi-stream models combine RGB and brightness (luminance) information through parallel processing pathways:

1. **BaseMultiChannelNetwork**: A dense/fully-connected model with separate pathways for RGB and brightness
2. **MultiChannelResNetNetwork**: A CNN-based model using ResNet architecture with separate pathways

Both models use the same unified augmentation pipeline, regularization techniques, and training strategy as the baseline for fair comparison.

### Advantages of Multi-Stream Processing
- Separation of color and brightness information allows each pathway to specialize
- The network can learn which stream is more informative for specific classes
- Potential for improved robustness to variations in lighting conditions

In [ ]:
# Create Multi-Stream Models
print("🏗️ Creating Multi-Stream Neural Network Models...")

# Check GPU availability and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ Using device: {device}")

if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("   Using CPU (CUDA not available)")

# Model configuration based on CIFAR-100 data
print(f"\n📊 Model Configuration:")
print(f"   Image size: 32x32 pixels")
print(f"   RGB channels: 3")
print(f"   Brightness channels: 1") 
print(f"   Number of classes: 100 (CIFAR-100)")
print(f"   Device: {device}")

# Use the model factory imported in the main import cell
if MODEL_FACTORY_AVAILABLE:
    print("\n📋 Available model types:")
    available_model_types = list_available_models()
    for model_type in available_model_types:
        print(f"   ✅ {model_type}")
    
    factory_available = True
else:
    print(f"❌ Model factory not available")
    print("💡 Using direct model imports...")
    factory_available = False

# Model dimensions for CIFAR-100
input_channels_rgb = 3
input_channels_brightness = 1  
image_size = 32
num_classes = 100

# For dense models: flatten the image to 1D
rgb_input_size = input_channels_rgb * image_size * image_size  # 3 * 32 * 32 = 3072
brightness_input_size = input_channels_brightness * image_size * image_size  # 1 * 32 * 32 = 1024

print(f"\n🔧 Model Input Configuration:")
print(f"   RGB input size (dense): {rgb_input_size}")
print(f"   Brightness input size (dense): {brightness_input_size}")
print(f"   RGB input channels (CNN): {input_channels_rgb}")
print(f"   Brightness input channels (CNN): {input_channels_brightness}")

# Create base_multi_channel_large (Dense/FC model)
print(f"\n🏭 Creating base_multi_channel_large (Dense Model)...")
try:
    if factory_available:
        base_multi_channel_large_model = create_model(
            'base_multi_channel_large',
            color_input_size=rgb_input_size,
            brightness_input_size=brightness_input_size,
            num_classes=num_classes,
            use_shared_classifier=True,
            device=device  # Use detected device (CUDA if available)
        )
    else:
        base_multi_channel_large_model = base_multi_channel_large(
            color_input_size=rgb_input_size,
            brightness_input_size=brightness_input_size,
            num_classes=num_classes,
            use_shared_classifier=True,
            device=device  # Use detected device (CUDA if available)
        )
    
    # Count parameters
    large_dense_params = sum(p.numel() for p in base_multi_channel_large_model.parameters())
    large_dense_trainable = sum(p.numel() for p in base_multi_channel_large_model.parameters() if p.requires_grad)
    
    print(f"✅ base_multi_channel_large created successfully")
    print(f"   Architecture: Large Dense/FC Network")
    print(f"   Total parameters: {large_dense_params:,}")
    print(f"   Trainable parameters: {large_dense_trainable:,}")
    print(f"   Input size: RGB {rgb_input_size}, Brightness {brightness_input_size}")
    print(f"   Fusion strategy: Shared classifier")
    print(f"   Device: {base_multi_channel_large_model.device}")
    
except Exception as e:
    print(f"❌ Failed to create base_multi_channel_large: {e}")
    print(f"💡 Error details: {str(e)}")
    traceback.print_exc()
    base_multi_channel_large_model = None

# Create multi_channel_resnet50 (CNN model)
print(f"\n🏭 Creating multi_channel_resnet50 (CNN Model)...")
try:
    if factory_available:
        multi_channel_resnet50_model = create_model(
            'multi_channel_resnet50',
            color_input_channels=input_channels_rgb,
            brightness_input_channels=input_channels_brightness,
            num_classes=num_classes,
            use_shared_classifier=True,
            activation='relu',
            device=device  # Use detected device (CUDA if available)
        )
    else:
        multi_channel_resnet50_model = multi_channel_resnet50(
            color_input_channels=input_channels_rgb,
            brightness_input_channels=input_channels_brightness,
            num_classes=num_classes,
            use_shared_classifier=True,
            activation='relu',
            device=device  # Use detected device (CUDA if available)
        )
    
    # Count parameters
    resnet50_params = sum(p.numel() for p in multi_channel_resnet50_model.parameters())
    resnet50_trainable = sum(p.numel() for p in multi_channel_resnet50_model.parameters() if p.requires_grad)
    
    print(f"✅ multi_channel_resnet50 created successfully")
    print(f"   Architecture: ResNet-50 style CNN (3,4,6,3 blocks)")
    print(f"   Total parameters: {resnet50_params:,}")
    print(f"   Trainable parameters: {resnet50_trainable:,}")
    print(f"   Input shape: RGB {(input_channels_rgb, image_size, image_size)}, Brightness {(input_channels_brightness, image_size, image_size)}")
    print(f"   Fusion strategy: Shared classifier")
    print(f"   Device: {multi_channel_resnet50_model.device}")
    
except Exception as e:
    print(f"❌ Failed to create multi_channel_resnet50: {e}")
    print(f"💡 Error details: {str(e)}")
    traceback.print_exc()
    multi_channel_resnet50_model = None

# Model comparison
if base_multi_channel_large_model is not None and multi_channel_resnet50_model is not None:
    print(f"\n📈 Model Comparison:")
    print(f"   base_multi_channel_large: {large_dense_params:,} parameters")
    print(f"   multi_channel_resnet50: {resnet50_params:,} parameters")
    print(f"   ResNet-50 is {resnet50_params/large_dense_params:.1f}x larger than Large Dense")
    
# Compile the models with proper optimizers
if base_multi_channel_large_model is not None:
    print("\n🔧 Compiling base_multi_channel_large model...")
    base_multi_channel_large_model.compile(
        optimizer='adamw',
        learning_rate=0.001,
        weight_decay=0.0001,
        loss='cross_entropy',
        metrics=['accuracy']
    )

if multi_channel_resnet50_model is not None:
    print("\n🔧 Compiling multi_channel_resnet50 model...")
    multi_channel_resnet50_model.compile(
        optimizer='adamw',
        learning_rate=0.001,
        weight_decay=0.0001,
        loss='cross_entropy',
        metrics=['accuracy']
    )
    
print("\n✅ Multi-stream models created and compiled successfully!")

## 14. Train Multi-Stream Models

In this section, we'll train our multi-stream neural network models using two approaches:

1. **Direct DataLoader Training (Primary Method)**
   - Using the `fit_dataloader()` method to pass DataLoaders directly
   - This enables on-the-fly augmentation during training
   - The model receives freshly augmented data in each epoch
   - More memory efficient as it doesn't require pre-extracting all augmented data

2. **Pre-extracted Data Training (Fallback Method)**
   - Using the `fit()` method with pre-extracted data arrays
   - Falls back to this approach if direct DataLoader training fails
   - Still uses augmented data, but extracted once before training

Both models (BaseMultiChannelNetwork and MultiChannelResNetNetwork) support these training methods, allowing for a consistent approach across all architectures.

### Benefits of Direct DataLoader Training
- Truly on-the-fly augmentation, creating different augmentations each epoch
- More memory efficient (doesn't store all augmented samples)
- Consistent with how the baseline ResNet model is trained
- Better generalization through exposure to more augmented variations

The training code includes error handling to fall back to the original method if needed.

In [ ]:
# Train base_multi_channel_large model using built-in fit_dataloader() API
print("🏋️‍♀️ Training BaseMultiChannelNetwork model using DataLoaders for on-the-fly augmentation...")

# Training configuration for all models
num_epochs = 100  # Full training run with early stopping
early_stopping_patience = 10

print(f"\n🔧 Training Configuration for BaseMultiChannelNetwork:")
print(f"   Epochs: {num_epochs} (with early stopping, patience={early_stopping_patience})")
print(f"   Batch size: 64")  # Using 64 as batch size to prevent CUDA errors
print(f"   Optimizer: AdamW with weight decay=0.0001")
print(f"   Learning rate: 0.001 with scheduler")
print(f"   Device: {base_multi_channel_large_model.device}")
print(f"   Training Method: fit_dataloader with on-the-fly augmentation")

# Clear any existing tqdm bars
try:
    tqdm._instances.clear()
except:
    pass

# Train using fit_dataloader() API which takes DataLoaders directly
print("\n🚀 Starting training with fit_dataloader() API for on-the-fly augmentation...")
try:
    base_multi_channel_large_history = base_multi_channel_large_model.fit_dataloader(
        train_loader=train_loader,  # Pass DataLoader directly
        val_loader=val_loader,      # Pass DataLoader directly
        epochs=num_epochs,
        early_stopping_patience=early_stopping_patience,
        scheduler_type='cosine',
        min_lr=1e-6,
        verbose=1,
        learning_rate=0.001,
        weight_decay=0.0001
    )
except Exception as e:
    print(f"⚠️ Encountered an error: {e}")
    print("🔍 Error details:")
    import traceback
    traceback.print_exc()
    
    print("\n🔄 Trying alternative approach...")
    # If the fit_dataloader method isn't working correctly, fall back to the original approach
    
    # Extract data from data loaders for fit() method
    # Get train data
    train_color_tensors = []
    train_brightness_tensors = []
    train_label_tensors = []

    print("\n🔄 Extracting train data from DataLoaders...")
    for data in train_loader:
        # Handle different data formats - support both tuple and list types
        if isinstance(data, (list, tuple)) and len(data) == 3:
            rgb, brightness, labels = data
        else:
            # Handle unexpected data format
            if not isinstance(data, (list, tuple)):
                raise ValueError(f"Expected data to be list or tuple, got {type(data)}")
            elif len(data) != 3:
                raise ValueError(f"Expected data to have 3 elements, got {len(data)}")
            else:
                raise ValueError("Unexpected data format from train_loader")
            
        train_color_tensors.append(rgb)
        train_brightness_tensors.append(brightness)
        train_label_tensors.append(labels)

    train_color_data = torch.cat(train_color_tensors, dim=0).cpu().numpy()
    train_brightness_data = torch.cat(train_brightness_tensors, dim=0).cpu().numpy()
    train_labels = torch.cat(train_label_tensors, dim=0).cpu().numpy()

    # Get validation data
    val_color_tensors = []
    val_brightness_tensors = []
    val_label_tensors = []

    print("🔄 Extracting validation data from DataLoaders...")
    for data in val_loader:
        # Handle different data formats - support both tuple and list types
        if isinstance(data, (list, tuple)) and len(data) == 3:
            rgb, brightness, labels = data
        else:
            # Handle unexpected data format
            if not isinstance(data, (list, tuple)):
                raise ValueError(f"Expected data to be list or tuple, got {type(data)}")
            elif len(data) != 3:
                raise ValueError(f"Expected data to have 3 elements, got {len(data)}")
            else:
                raise ValueError("Unexpected data format from val_loader")
            
        val_color_tensors.append(rgb)
        val_brightness_tensors.append(brightness)
        val_label_tensors.append(labels)

    val_color_data = torch.cat(val_color_tensors, dim=0).cpu().numpy()
    val_brightness_data = torch.cat(val_brightness_tensors, dim=0).cpu().numpy()
    val_labels = torch.cat(val_label_tensors, dim=0).cpu().numpy()

    print(f"✅ Extracted data shapes:")
    print(f"   Train color: {train_color_data.shape}, Train brightness: {train_brightness_data.shape}, Train labels: {train_labels.shape}")
    print(f"   Val color: {val_color_data.shape}, Val brightness: {val_brightness_data.shape}, Val labels: {val_labels.shape}")

    # IMPORTANT: Reshape the data correctly for the model - this fixes the dimension mismatch
    # The dense model expects flattened inputs (N, C*H*W) rather than images (N, C, H, W)
    print("\n🔄 Reshaping data for dense model...")

    # Get the expected input sizes from the model definition
    rgb_input_size = base_multi_channel_large_model.color_input_size  # Should be 3072 for CIFAR-100 (3*32*32)
    brightness_input_size = base_multi_channel_large_model.brightness_input_size  # Should be 1024 for CIFAR-100 (1*32*32)

    # Verify the expected shapes
    print(f"   Expected RGB input size: {rgb_input_size}")
    print(f"   Expected brightness input size: {brightness_input_size}")

    # Reshape RGB data from (N, C, H, W) to (N, C*H*W)
    train_color_data_reshaped = train_color_data.reshape(train_color_data.shape[0], -1)
    val_color_data_reshaped = val_color_data.reshape(val_color_data.shape[0], -1)

    # Reshape brightness data from (N, C, H, W) to (N, C*H*W)
    train_brightness_data_reshaped = train_brightness_data.reshape(train_brightness_data.shape[0], -1)
    val_brightness_data_reshaped = val_brightness_data.reshape(val_brightness_data.shape[0], -1)

    # Verify the reshaped dimensions match what the model expects
    print(f"   Reshaped train color: {train_color_data_reshaped.shape} (should be [N, {rgb_input_size}])")
    print(f"   Reshaped train brightness: {train_brightness_data_reshaped.shape} (should be [N, {brightness_input_size}])")
    print(f"   Reshaped val color: {val_color_data_reshaped.shape} (should be [N, {rgb_input_size}])")
    print(f"   Reshaped val brightness: {val_brightness_data_reshaped.shape} (should be [N, {brightness_input_size}])")

    # Double-check that shapes match exactly what's expected
    if train_color_data_reshaped.shape[1] != rgb_input_size:
        print(f"⚠️ Warning: Reshaped RGB size {train_color_data_reshaped.shape[1]} doesn't match expected size {rgb_input_size}")
    if train_brightness_data_reshaped.shape[1] != brightness_input_size:
        print(f"⚠️ Warning: Reshaped brightness size {train_brightness_data_reshaped.shape[1]} doesn't match expected size {brightness_input_size}")

    # Clear any existing tqdm bars
    try:
        tqdm._instances.clear()
    except:
        pass

    # Train using fit() API which takes numpy arrays directly
    print("\n🚀 Starting training with fit() API as fallback...")
    base_multi_channel_large_history = base_multi_channel_large_model.fit(
        train_color_data=train_color_data_reshaped,  # Use reshaped data
        train_brightness_data=train_brightness_data_reshaped,  # Use reshaped data
        train_labels=train_labels,
        val_color_data=val_color_data_reshaped,  # Use reshaped data
        val_brightness_data=val_brightness_data_reshaped,  # Use reshaped data
        val_labels=val_labels,
        batch_size=64,  # Using 64 as batch size to prevent CUDA errors
        epochs=num_epochs,
        early_stopping_patience=early_stopping_patience,
        verbose=1,
        num_workers=0,  # Reduce worker processes to avoid CUDA initialization issues
        pin_memory=False  # Try disabling pin_memory which can cause issues with CUDA
    )

# Plot training history
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(base_multi_channel_large_history['accuracy'], label='Train')
plt.plot(base_multi_channel_large_history['val_accuracy'], label='Validation')
plt.title('BaseMultiChannelNetwork Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(base_multi_channel_large_history['loss'], label='Train')
plt.plot(base_multi_channel_large_history['val_loss'], label='Validation')
plt.title('BaseMultiChannelNetwork Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ BaseMultiChannelNetwork training complete!")

In [ ]:
# Train multi_channel_resnet50 model using built-in fit_dataloader() API
print("🏋️‍♀️ Training MultiChannelResNetNetwork model using DataLoaders for on-the-fly augmentation...")

print(f"\n🔧 Training Configuration for MultiChannelResNetNetwork:")
print(f"   Epochs: {num_epochs} (with early stopping, patience={early_stopping_patience})")
print(f"   Batch size: 512")  # Increased batch size for A100 GPU
print(f"   Optimizer: AdamW with weight decay=0.0001")
print(f"   Learning rate: 0.001 with scheduler")
print(f"   Device: {multi_channel_resnet50_model.device}")
print(f"   Training Method: fit_dataloader with on-the-fly augmentation")

# Make sure CUDA_LAUNCH_BLOCKING is still set
if 'CUDA_LAUNCH_BLOCKING' not in os.environ:
    os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Clear any existing tqdm bars
try:
    tqdm._instances.clear()
except:
    pass

# Train using fit_dataloader() API which takes DataLoaders directly
print("\n🚀 Starting training with fit_dataloader() API for on-the-fly augmentation...")
try:
    multi_channel_resnet50_history = multi_channel_resnet50_model.fit_dataloader(
        train_loader=train_loader,  # Pass DataLoader directly
        val_loader=val_loader,      # Pass DataLoader directly
        epochs=num_epochs,
        early_stopping_patience=early_stopping_patience,
        scheduler_type='cosine',
        min_lr=1e-6,
        verbose=1,
        learning_rate=0.001,
        weight_decay=0.0001
    )
except Exception as e:
    print(f"⚠️ Encountered an error: {e}")
    print("🔍 Error details:")
    import traceback
    traceback.print_exc()
    
    print("\n🔍 Data consistency check...")
    print(f"   We have two sets of data in the notebook:")
    print(f"   1. Original processed data: RGB: {train_rgb.shape}, Brightness: {train_brightness.shape}, Labels: {train_labels_tensor.shape}")
    print(f"   2. Augmented data from DataLoaders: RGB: {train_color_data.shape}, Brightness: {train_brightness_data.shape}, Labels: {train_labels.shape}")
    
    # Falling back to the original approach using extracted arrays
    print("\n🔄 Trying alternative approach...")
    # If the fit_dataloader method isn't working correctly, fall back to the original approach
    
    # IMPORTANT: Use the augmented data from DataLoaders for training
    # This data includes augmentations (flips, rotations, color jitter, etc.)
    cnn_train_color = torch.tensor(train_color_data)  # Augmented RGB data
    cnn_train_brightness = torch.tensor(train_brightness_data)  # Augmented brightness data
    cnn_train_labels = train_labels  # Augmented labels
    cnn_val_color = torch.tensor(val_color_data)  # Validation RGB data
    cnn_val_brightness = torch.tensor(val_brightness_data)  # Validation brightness data
    cnn_val_labels = val_labels  # Validation labels

    print("\n🔍 Using augmented data from DataLoaders for better training:")
    print(f"   Training - RGB: {cnn_train_color.shape}, Brightness: {cnn_train_brightness.shape}, Labels: {cnn_train_labels.shape}")
    print(f"   Validation - RGB: {cnn_val_color.shape}, Brightness: {cnn_val_brightness.shape}, Labels: {cnn_val_labels.shape}")
    print(f"   ✅ Using augmented data with {cnn_train_color.shape[0]} samples (vs. {train_rgb.shape[0]} in original data)")

    # Train using fit() API which takes numpy arrays directly
    print("\n🚀 Starting training with fit() API as fallback...")
    try:
        multi_channel_resnet50_history = multi_channel_resnet50_model.fit(
            train_color_data=cnn_train_color.numpy(),
            train_brightness_data=cnn_train_brightness.numpy(),
            train_labels=cnn_train_labels,
            val_color_data=cnn_val_color.numpy(),
            val_brightness_data=cnn_val_brightness.numpy(),
            val_labels=cnn_val_labels,
            batch_size=512,  # Increased batch size for A100 GPU
            epochs=num_epochs,
            early_stopping_patience=early_stopping_patience,
            verbose=1,
            num_workers=0,  # Reduce worker processes to avoid CUDA initialization issues
            pin_memory=False  # Try disabling pin_memory which can cause issues with CUDA
        )
    except RuntimeError as e:
        print(f"⚠️ Encountered a CUDA error: {e}")
        print("🔍 Attempting with alternative configuration...")
        
        # Try with data already on the GPU
        device = multi_channel_resnet50_model.device
        
        # Convert data to tensors on the GPU (using CNN format)
        train_color_tensor = cnn_train_color.to(device)
        train_brightness_tensor = cnn_train_brightness.to(device)
        val_color_tensor = cnn_val_color.to(device)
        val_brightness_tensor = cnn_val_brightness.to(device)
        train_labels_tensor = torch.tensor(cnn_train_labels, device=device, dtype=torch.long)
        val_labels_tensor = torch.tensor(cnn_val_labels, device=device, dtype=torch.long)
        
        # Clear any existing tqdm bars again
        try:
            for inst in list(getattr(tqdm, '_instances', [])):
                try:
                    inst.close()
                except Exception:
                    pass
        except Exception:
            pass
            
        # Try with direct tensor input instead of numpy arrays
        multi_channel_resnet50_history = multi_channel_resnet50_model.fit(
            train_color_data=train_color_tensor,
            train_brightness_data=train_brightness_tensor,
            train_labels=train_labels_tensor,
            val_color_data=val_color_tensor,
            val_brightness_data=val_brightness_tensor,
            val_labels=val_labels_tensor,
            batch_size=512,  # Increased batch size for A100 GPU
            epochs=num_epochs,
            early_stopping_patience=early_stopping_patience,
            verbose=1,
            num_workers=0,
            pin_memory=False
        )
    
# Plot training history
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(multi_channel_resnet50_history['accuracy'], label='Train')
plt.plot(multi_channel_resnet50_history['val_accuracy'], label='Validation')
plt.title('MultiChannelResNetNetwork Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(multi_channel_resnet50_history['loss'], label='Train')
plt.plot(multi_channel_resnet50_history['val_loss'], label='Validation')
plt.title('MultiChannelResNetNetwork Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ CNN model training complete!")

## 15. Evaluate Models

Evaluate the trained models on the test set and compare their performance.

In [ ]:
# Evaluate Models
print("📊 Evaluating models on the test set...")

# Evaluate baseline model (which doesn't have our API)
def evaluate_baseline_model(model, test_loader, criterion, model_name="Model"):
    """
    Evaluate the baseline ResNet model on the test set.
    
    Args:
        model: The model to evaluate
        test_loader: DataLoader for test data
        criterion: Loss function
        model_name: Name for logging
        
    Returns:
        Dictionary with evaluation metrics
    """
    device = next(model.parameters()).device
    model.eval()
    
    test_loss = 0.0
    test_correct = 0
    test_total = 0
    
    all_predictions = []
    all_targets = []
    
    with torch.no_grad():
        test_bar = tqdm(test_loader, desc=f"{model_name} Testing")
        
        for batch_idx, data in enumerate(test_bar):
            # Handle different data formats - support both tuple and list types
            if isinstance(data, (list, tuple)) and len(data) == 3:
                rgb, _, targets = data  # Baseline only uses RGB
            else:
                # Handle unexpected data format
                if not isinstance(data, (list, tuple)):
                    raise ValueError(f"Expected data to be list or tuple, got {type(data)}")
                elif len(data) != 3:
                    raise ValueError(f"Expected data to have 3 elements, got {len(data)}")
                else:
                    rgb, targets = data
                
            # Move to device
            rgb = rgb.to(device)
            targets = targets.to(device)
            
            # Baseline model only takes RGB
            outputs = model(rgb)
            loss = criterion(outputs, targets)
            
            # Track statistics
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            test_total += targets.size(0)
            test_correct += predicted.eq(targets).sum().item()
            
            # Store predictions and targets for detailed metrics
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
            
            # Update progress bar
            test_bar.set_postfix({
                'loss': test_loss/(batch_idx+1), 
                'acc': 100.*test_correct/test_total
            })
    
    test_acc = 100. * test_correct / test_total
    test_loss = test_loss / len(test_loader)
    
    # Print summary
    print(f"\n📈 {model_name} Test Results:")
    print(f"   Test Loss: {test_loss:.4f}")
    print(f"   Test Accuracy: {test_acc:.2f}%")
    
    # Generate classification report - using sklearn.metrics imported in the main import cell
    report = classification_report(
        all_targets, 
        all_predictions, 
        target_names=[CIFAR100_FINE_LABELS[i] for i in range(100)],
        output_dict=True
    )
    
    return {
        'test_loss': test_loss,
        'test_acc': test_acc,
        'classification_report': report,
        'predictions': all_predictions,
        'targets': all_targets
    }

# Evaluate baseline model (doesn't have our API)
baseline_results = evaluate_baseline_model(
    model=baseline_model,
    test_loader=test_loader,
    criterion=baseline_criterion,
    model_name="Baseline ResNet50"
)

# Extract test data for evaluation using our API
test_color_tensors = []
test_brightness_tensors = []
test_label_tensors = []

# Safely extract data from test_loader, handling both tuple and list formats
for data in test_loader:
    # Handle different data formats - support both tuple and list types
    if isinstance(data, (list, tuple)) and len(data) == 3:
        rgb, brightness, labels = data
    else:
        # Handle unexpected data format
        if not isinstance(data, (list, tuple)):
            raise ValueError(f"Expected data to be list or tuple, got {type(data)}")
        elif len(data) != 3:
            raise ValueError(f"Expected data to have 3 elements, got {len(data)}")
        else:
            raise ValueError("Unexpected data format from test_loader")
    
    test_color_tensors.append(rgb)
    test_brightness_tensors.append(brightness)
    test_label_tensors.append(labels)

test_color_data = torch.cat(test_color_tensors, dim=0).cpu().numpy()
test_brightness_data = torch.cat(test_brightness_tensors, dim=0).cpu().numpy()
test_labels = torch.cat(test_label_tensors, dim=0).cpu().numpy()

# Reshape the data for dense models
print("\n🔄 Preparing test data for evaluation...")
test_color_data_reshaped = test_color_data.reshape(test_color_data.shape[0], -1)
test_brightness_data_reshaped = test_brightness_data.reshape(test_brightness_data.shape[0], -1)

print(f"   Original RGB test data shape: {test_color_data.shape}")
print(f"   Original brightness test data shape: {test_brightness_data.shape}")
print(f"   Reshaped RGB test data shape: {test_color_data_reshaped.shape}")
print(f"   Reshaped brightness test data shape: {test_brightness_data_reshaped.shape}")

# Evaluate BaseMultiChannelNetwork (use reshaped data for dense model)
print("\n📊 Evaluating multi-stream models using built-in API...")
print("   Evaluating BaseMultiChannelNetwork (dense model)...")

# Verify the input shapes match the model expectations
base_model_rgb_size = base_multi_channel_large_model.color_input_size
base_model_brightness_size = base_multi_channel_large_model.brightness_input_size

if test_color_data_reshaped.shape[1] != base_model_rgb_size:
    print(f"⚠️ Warning: Test RGB shape {test_color_data_reshaped.shape[1]} doesn't match model expected size {base_model_rgb_size}")
if test_brightness_data_reshaped.shape[1] != base_model_brightness_size:
    print(f"⚠️ Warning: Test brightness shape {test_brightness_data_reshaped.shape[1]} doesn't match model expected size {base_model_brightness_size}")

# Evaluate BaseMultiChannelNetwork using reshaped data
base_multi_channel_large_results = base_multi_channel_large_model.evaluate(
    test_color_data=test_color_data_reshaped,  # Use reshaped (flattened) data for dense model
    test_brightness_data=test_brightness_data_reshaped,  # Use reshaped data
    test_labels=test_labels,
    batch_size=512  # Increased batch size for A100 GPU
)

# Convert results for consistency with baseline format
base_multi_channel_large_results['test_acc'] = base_multi_channel_large_results['accuracy'] * 100

# Check if the ResNet model is CNN-based
is_resnet_cnn = hasattr(multi_channel_resnet50_model, 'is_cnn') and multi_channel_resnet50_model.is_cnn
if not hasattr(multi_channel_resnet50_model, 'is_cnn'):
    is_resnet_cnn = 'resnet' in multi_channel_resnet50_model.__class__.__name__.lower()

# Choose the appropriate data format for ResNet model
print(f"   Evaluating MultiChannelResNetNetwork ({'CNN' if is_resnet_cnn else 'dense'} model)...")

if is_resnet_cnn:
    # Use original shape for CNN models
    resnet_test_color = test_color_data
    resnet_test_brightness = test_brightness_data
    print(f"   Using original image format (N,C,H,W) for CNN model")
else:
    # Use reshaped data for dense models
    resnet_test_color = test_color_data_reshaped
    resnet_test_brightness = test_brightness_data_reshaped
    print(f"   Using reshaped format (N,C*H*W) for dense model")

# Evaluate MultiChannelResNetNetwork with appropriate data format
multi_channel_resnet50_results = multi_channel_resnet50_model.evaluate(
    test_color_data=resnet_test_color,
    test_brightness_data=resnet_test_brightness,
    test_labels=test_labels,
    batch_size=512  # Increased batch size for A100 GPU
)

# Convert results for consistency with baseline format
multi_channel_resnet50_results['test_acc'] = multi_channel_resnet50_results['accuracy'] * 100

# Compare models
print("\n🔍 Model Comparison on Test Set:")
print(f"   Baseline ResNet50: {baseline_results['test_acc']:.2f}%")
print(f"   BaseMultiChannelNetwork: {base_multi_channel_large_results['test_acc']:.2f}%")
print(f"   MultiChannelResNetNetwork: {multi_channel_resnet50_results['test_acc']:.2f}%")

# Calculate improvement over baseline
base_improvement = base_multi_channel_large_results['test_acc'] - baseline_results['test_acc']
resnet_improvement = multi_channel_resnet50_results['test_acc'] - baseline_results['test_acc']

print(f"\n📊 Improvement over Baseline:")
print(f"   BaseMultiChannelNetwork: {base_improvement:.2f}% points")
print(f"   MultiChannelResNetNetwork: {resnet_improvement:.2f}% points")

# Visualize training curves comparison for all models
plt.figure(figsize=(14, 8))

# Convert baseline history keys to match multi-stream model history keys
baseline_history_converted = {
    'train_accuracy': [acc/100 for acc in baseline_history['train_acc']],
    'val_accuracy': [acc/100 for acc in baseline_history['val_acc']],
    'train_loss': baseline_history['train_loss'],
    'val_loss': baseline_history['val_loss']
}

# Accuracy curves
plt.subplot(1, 2, 1)
plt.plot(baseline_history_converted['train_accuracy'], label='Baseline Train')
plt.plot(baseline_history_converted['val_accuracy'], label='Baseline Val')
plt.plot(base_multi_channel_large_history['train_accuracy'], label='Dense Multi-Stream Train')
plt.plot(base_multi_channel_large_history['val_accuracy'], label='Dense Multi-Stream Val')
plt.plot(multi_channel_resnet50_history['train_accuracy'], label='ResNet Multi-Stream Train')
plt.plot(multi_channel_resnet50_history['val_accuracy'], label='ResNet Multi-Stream Val')
plt.title('Model Accuracy Comparison')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

# Loss curves
plt.subplot(1, 2, 2)
plt.plot(baseline_history_converted['train_loss'], label='Baseline Train')
plt.plot(baseline_history_converted['val_loss'], label='Baseline Val')
plt.plot(base_multi_channel_large_history['train_loss'], label='Dense Multi-Stream Train')
plt.plot(base_multi_channel_large_history['val_loss'], label='Dense Multi-Stream Val')
plt.plot(multi_channel_resnet50_history['train_loss'], label='ResNet Multi-Stream Train')
plt.plot(multi_channel_resnet50_history['val_loss'], label='ResNet Multi-Stream Val')
plt.title('Model Loss Comparison')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Additional pathway analysis for multi-stream models
print("\n🔬 Multi-Stream Pathway Analysis:")

# Analyze pathway importance for multi-stream models
base_pathway_importance = base_multi_channel_large_model.analyze_pathway_weights()
resnet_pathway_importance = multi_channel_resnet50_model.analyze_pathway_weights()

print("BaseMultiChannelNetwork Pathway Importance:")
for key, value in base_pathway_importance.items():
    print(f"   {key}: {value:.4f}")
    
print("\nMultiChannelResNetNetwork Pathway Importance:")
for key, value in resnet_pathway_importance.items():
    print(f"   {key}: {value:.4f}")

print("\n✅ Model evaluation complete!")

# Summarize overfitting analysis
print("\n🔍 Overfitting Analysis:")
for model_name, history in [
    ("Baseline ResNet50", baseline_history_converted),
    ("BaseMultiChannelNetwork", base_multi_channel_large_history),
    ("MultiChannelResNetNetwork", multi_channel_resnet50_history)
]:
    # Convert to numpy arrays for calculations, handling different key formats
    if 'train_acc' in history:
        train_acc = np.array(history['train_acc'])
        val_acc = np.array(history['val_acc'])
    else:
        train_acc = np.array(history['train_accuracy']) * 100  # Convert to percentage
        val_acc = np.array(history['val_accuracy']) * 100  # Convert to percentage
    
    gap = train_acc - val_acc
    last_gap = gap[-1]
    max_gap = np.max(gap)
    avg_last_5 = np.mean(gap[-5:]) if len(gap) >= 5 else np.mean(gap)
    
    print(f"\n   {model_name}:")
    print(f"      Final train-val gap: {last_gap:.2f}%")
    print(f"      Maximum gap during training: {max_gap:.2f}%")
    print(f"      Average gap in last 5 epochs: {avg_last_5:.2f}%")
    
    # Evaluate overfitting level
    if avg_last_5 < 3:
        print(f"      ✅ No significant overfitting (gap < 3%)")
    elif avg_last_5 < 7:
        print(f"      ⚠️ Mild overfitting (3% ≤ gap < 7%)")
    elif avg_last_5 < 15:
        print(f"      🔴 Moderate overfitting (7% ≤ gap < 15%)")
    else:
        print(f"      ❌ Severe overfitting (gap ≥ 15%)")

## 16. Pathway Analysis

Analyze the contribution of each pathway (RGB and brightness) to the model's predictions.

In [ ]:
# Pathway Analysis
print("🔍 Analyzing pathway contributions for multi-stream models...")

def analyze_pathways(model, test_loader, num_samples=100, model_name="Model"):
    """
    Analyze contributions of RGB and brightness pathways.
    
    Args:
        model: The multi-stream model to analyze
        test_loader: DataLoader for test data
        num_samples: Number of samples to analyze
        model_name: Name for logging
        
    Returns:
        Dictionary with pathway analysis results
    """
    device = next(model.parameters()).device
    model.eval()
    
    # Check if model is CNN-based or dense
    is_cnn_model = hasattr(model, 'is_cnn') and model.is_cnn
    if not hasattr(model, 'is_cnn'):
        is_cnn_model = 'resnet' in model.__class__.__name__.lower()
    
    print(f"   Model type: {'CNN' if is_cnn_model else 'Dense'}")
    
    all_rgb = []
    all_brightness = []
    all_targets = []
    all_combined_outputs = []
    all_rgb_outputs = []
    all_brightness_outputs = []
    
    sample_count = 0
    
    with torch.no_grad():
        for data in test_loader:
            # Break when we have enough samples
            if sample_count >= num_samples:
                break
            
            # Handle different data formats - support both tuple and list types
            if isinstance(data, (list, tuple)) and len(data) == 3:
                rgb, brightness, targets = data
            else:
                # Handle unexpected data format
                if not isinstance(data, (list, tuple)):
                    raise ValueError(f"Expected data to be list or tuple, got {type(data)}")
                elif len(data) != 3:
                    raise ValueError(f"Expected data to have 3 elements, got {len(data)}")
                else:
                    raise ValueError("Unexpected data format from test_loader")
            
            # Collect only the samples we need
            remaining = num_samples - sample_count
            if remaining < len(rgb):
                rgb = rgb[:remaining]
                brightness = brightness[:remaining]
                targets = targets[:remaining]
            
            # Move to device
            rgb, brightness, targets = rgb.to(device), brightness.to(device), targets.to(device)
            
            # For dense models, we need to reshape the data
            if not is_cnn_model:
                # Reshape (N, C, H, W) to (N, C*H*W)
                rgb_reshaped = rgb.reshape(rgb.size(0), -1)
                brightness_reshaped = brightness.reshape(brightness.size(0), -1)
                
                # Get outputs from combined and individual pathways
                combined_outputs = model(rgb_reshaped, brightness_reshaped)
                
                # Use the analyze_pathways method to get individual pathway outputs
                rgb_outputs, brightness_outputs = model.analyze_pathways(rgb_reshaped, brightness_reshaped)
            else:
                # CNN models use the original shape
                combined_outputs = model(rgb, brightness)
                rgb_outputs, brightness_outputs = model.analyze_pathways(rgb, brightness)
            
            all_rgb.append(rgb.cpu())
            all_brightness.append(brightness.cpu())
            all_targets.append(targets.cpu())
            all_combined_outputs.append(combined_outputs.cpu())
            all_rgb_outputs.append(rgb_outputs.cpu())
            all_brightness_outputs.append(brightness_outputs.cpu())
            
            sample_count += len(rgb)
    
    # Concatenate all data
    all_rgb = torch.cat(all_rgb)
    all_brightness = torch.cat(all_brightness)
    all_targets = torch.cat(all_targets)
    all_combined_outputs = torch.cat(all_combined_outputs)
    all_rgb_outputs = torch.cat(all_rgb_outputs)
    all_brightness_outputs = torch.cat(all_brightness_outputs)
    
    # Calculate accuracy for each pathway
    _, combined_preds = all_combined_outputs.max(1)
    _, rgb_preds = all_rgb_outputs.max(1)
    _, brightness_preds = all_brightness_outputs.max(1)
    
    combined_acc = 100. * (combined_preds == all_targets).sum().item() / len(all_targets)
    rgb_acc = 100. * (rgb_preds == all_targets).sum().item() / len(all_targets)
    brightness_acc = 100. * (brightness_preds == all_targets).sum().item() / len(all_targets)
    
    print(f"\n📊 {model_name} Pathway Analysis:")
    print(f"   Combined accuracy: {combined_acc:.2f}%")
    print(f"   RGB pathway accuracy: {rgb_acc:.2f}%")
    print(f"   Brightness pathway accuracy: {brightness_acc:.2f}%")
    
    # Calculate pathway agreement
    rgb_brightness_agreement = 100. * (rgb_preds == brightness_preds).sum().item() / len(all_targets)
    combined_rgb_agreement = 100. * (combined_preds == rgb_preds).sum().item() / len(all_targets)
    combined_brightness_agreement = 100. * (combined_preds == brightness_preds).sum().item() / len(all_targets)
    
    print(f"\n🤝 Pathway Agreement:")
    print(f"   RGB-Brightness agreement: {rgb_brightness_agreement:.2f}%")
    print(f"   Combined-RGB agreement: {combined_rgb_agreement:.2f}%")
    print(f"   Combined-Brightness agreement: {combined_brightness_agreement:.2f}%")
    
    return {
        'combined_acc': combined_acc,
        'rgb_acc': rgb_acc,
        'brightness_acc': brightness_acc,
        'rgb_brightness_agreement': rgb_brightness_agreement,
        'combined_rgb_agreement': combined_rgb_agreement,
        'combined_brightness_agreement': combined_brightness_agreement
    }

# Analyze BaseMultiChannelNetwork
base_multi_channel_large_pathway_analysis = analyze_pathways(
    model=base_multi_channel_large_model,
    test_loader=test_loader,
    num_samples=500,  # Increased sample size for A100 GPU
    model_name="BaseMultiChannelNetwork"
)

# Analyze MultiChannelResNetNetwork
multi_channel_resnet50_pathway_analysis = analyze_pathways(
    model=multi_channel_resnet50_model,
    test_loader=test_loader,
    num_samples=500,  # Increased sample size for A100 GPU
    model_name="MultiChannelResNetNetwork"
)

# Visualize Model Predictions and Pathway Analysis
print("🔍 Visualizing model predictions and pathway analysis...")

# Get a batch of test data
test_batch = next(iter(test_loader))
test_rgb, test_brightness, test_labels = test_batch
test_rgb = test_rgb.to(device)
test_brightness = test_brightness.to(device)
test_labels = test_labels.to(device)

# Determine model types
base_is_cnn = hasattr(base_multi_channel_large_model, 'is_cnn') and base_multi_channel_large_model.is_cnn
if not hasattr(base_multi_channel_large_model, 'is_cnn'):
    base_is_cnn = 'resnet' in base_multi_channel_large_model.__class__.__name__.lower()

resnet_is_cnn = hasattr(multi_channel_resnet50_model, 'is_cnn') and multi_channel_resnet50_model.is_cnn
if not hasattr(multi_channel_resnet50_model, 'is_cnn'):
    resnet_is_cnn = 'resnet' in multi_channel_resnet50_model.__class__.__name__.lower()

# Get predictions from all models
with torch.no_grad():
    # Baseline model only takes RGB
    baseline_outputs = baseline_model(test_rgb)
    baseline_probs = torch.softmax(baseline_outputs, dim=1)
    baseline_preds = torch.argmax(baseline_probs, dim=1)
    
    # Multi-stream models need proper reshaping for dense models
    # For base_multi_channel_large_model (usually dense)
    if not base_is_cnn:
        # Reshape for dense model
        test_rgb_reshaped = test_rgb.reshape(test_rgb.size(0), -1)
        test_brightness_reshaped = test_brightness.reshape(test_brightness.size(0), -1)
        
        # Forward pass with reshaped data
        base_multi_channel_outputs = base_multi_channel_large_model(test_rgb_reshaped, test_brightness_reshaped)
        base_color_outputs, base_brightness_outputs = base_multi_channel_large_model.analyze_pathways(
            test_rgb_reshaped, test_brightness_reshaped
        )
    else:
        # CNN model uses original shape
        base_multi_channel_outputs = base_multi_channel_large_model(test_rgb, test_brightness)
        base_color_outputs, base_brightness_outputs = base_multi_channel_large_model.analyze_pathways(
            test_rgb, test_brightness
        )
    
    # For multi_channel_resnet50_model (usually CNN)
    if not resnet_is_cnn:
        # Reshape for dense model
        test_rgb_reshaped = test_rgb.reshape(test_rgb.size(0), -1)
        test_brightness_reshaped = test_brightness.reshape(test_brightness.size(0), -1)
        
        # Forward pass with reshaped data
        multi_channel_resnet_outputs = multi_channel_resnet50_model(test_rgb_reshaped, test_brightness_reshaped)
        resnet_color_outputs, resnet_brightness_outputs = multi_channel_resnet50_model.analyze_pathways(
            test_rgb_reshaped, test_brightness_reshaped
        )
    else:
        # CNN model uses original shape
        multi_channel_resnet_outputs = multi_channel_resnet50_model(test_rgb, test_brightness)
        resnet_color_outputs, resnet_brightness_outputs = multi_channel_resnet50_model.analyze_pathways(
            test_rgb, test_brightness
        )
    
    # Calculate probabilities and predictions
    base_multi_channel_probs = torch.softmax(base_multi_channel_outputs, dim=1)
    base_multi_channel_preds = torch.argmax(base_multi_channel_probs, dim=1)
    
    multi_channel_resnet_probs = torch.softmax(multi_channel_resnet_outputs, dim=1)
    multi_channel_resnet_preds = torch.argmax(multi_channel_resnet_probs, dim=1)
    
    base_color_probs = torch.softmax(base_color_outputs, dim=1)
    base_brightness_probs = torch.softmax(base_brightness_outputs, dim=1)
    base_color_preds = torch.argmax(base_color_probs, dim=1)
    base_brightness_preds = torch.argmax(base_brightness_probs, dim=1)
    
    resnet_color_probs = torch.softmax(resnet_color_outputs, dim=1)
    resnet_brightness_probs = torch.softmax(resnet_brightness_outputs, dim=1)
    resnet_color_preds = torch.argmax(resnet_color_probs, dim=1)
    resnet_brightness_preds = torch.argmax(resnet_brightness_probs, dim=1)

# Move to CPU for visualization
test_rgb = test_rgb.cpu()
test_brightness = test_brightness.cpu()
test_labels = test_labels.cpu()
baseline_preds = baseline_preds.cpu()
base_multi_channel_preds = base_multi_channel_preds.cpu()
multi_channel_resnet_preds = multi_channel_resnet_preds.cpu()
base_color_preds = base_color_preds.cpu()
base_brightness_preds = base_brightness_preds.cpu()
resnet_color_preds = resnet_color_preds.cpu()
resnet_brightness_preds = resnet_brightness_preds.cpu()

# Select a subset of samples for visualization
n_samples = min(8, test_rgb.size(0))
sample_indices = np.random.choice(test_rgb.size(0), n_samples, replace=False)

# Plot original images, model predictions, and pathway contributions
fig, axes = plt.subplots(n_samples, 6, figsize=(18, 3*n_samples))
if n_samples == 1:
    axes = axes.reshape(1, -1)

for i, idx in enumerate(sample_indices):
    # Get RGB image
    rgb_img = test_rgb[idx].permute(1, 2, 0).numpy()
    rgb_img = np.clip(rgb_img, 0, 1)
    
    # Get brightness image
    brightness_img = test_brightness[idx, 0].numpy()
    
    # True label and predictions
    true_label = test_labels[idx].item()
    baseline_pred = baseline_preds[idx].item()
    multi_channel_pred = multi_channel_resnet_preds[idx].item()
    color_pathway_pred = resnet_color_preds[idx].item()
    brightness_pathway_pred = resnet_brightness_preds[idx].item()
    
    # Plot original RGB image
    axes[i, 0].imshow(rgb_img)
    axes[i, 0].set_title(f"True: {CIFAR100_FINE_LABELS[true_label]}")
    axes[i, 0].axis('off')
    
    # Plot brightness image
    axes[i, 1].imshow(brightness_img, cmap='gray')
    axes[i, 1].set_title("Brightness Channel")
    axes[i, 1].axis('off')
    
    # Plot baseline model prediction
    axes[i, 2].imshow(rgb_img)
    correct = true_label == baseline_pred
    title_color = 'green' if correct else 'red'
    axes[i, 2].set_title(f"Baseline: {CIFAR100_FINE_LABELS[baseline_pred]}", color=title_color)
    axes[i, 2].axis('off')
    
    # Plot multi-stream model prediction
    axes[i, 3].imshow(rgb_img)
    correct = true_label == multi_channel_pred
    title_color = 'green' if correct else 'red'
    axes[i, 3].set_title(f"Multi-Stream: {CIFAR100_FINE_LABELS[multi_channel_pred]}", color=title_color)
    axes[i, 3].axis('off')
    
    # Plot color pathway prediction
    axes[i, 4].imshow(rgb_img)
    correct = true_label == color_pathway_pred
    title_color = 'green' if correct else 'red'
    axes[i, 4].set_title(f"Color Path: {CIFAR100_FINE_LABELS[color_pathway_pred]}", color=title_color)
    axes[i, 4].axis('off')
    
    # Plot brightness pathway prediction
    axes[i, 5].imshow(brightness_img, cmap='gray')
    correct = true_label == brightness_pathway_pred
    title_color = 'green' if correct else 'red'
    axes[i, 5].set_title(f"Brightness Path: {CIFAR100_FINE_LABELS[brightness_pathway_pred]}", color=title_color)
    axes[i, 5].axis('off')

plt.tight_layout()
plt.show()

# Visualize pathway feature importance
plt.figure(figsize=(12, 6))

# Get pathway importance
base_importance = base_multi_channel_large_model.get_pathway_importance()
resnet_importance = multi_channel_resnet50_model.get_pathway_importance()

# Plot importance values
labels = ['Color Pathway', 'Brightness Pathway']
base_values = [base_importance['color_pathway'], base_importance['brightness_pathway']]
resnet_values = [resnet_importance['color_pathway'], resnet_importance['brightness_pathway']]

x = np.arange(len(labels))
width = 0.35

plt.bar(x - width/2, base_values, width, label='BaseMultiChannelNetwork')
plt.bar(x + width/2, resnet_values, width, label='MultiChannelResNetNetwork')

plt.ylabel('Relative Importance')
plt.title('Pathway Importance Analysis')
plt.xticks(x, labels)
plt.ylim(0, 1)
plt.legend()
plt.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Visualization complete!")

## 17. Save Models

Save the trained models for later use.

In [ ]:
# Save Models
print("💾 Saving trained models...")

# Create models directory if it doesn't exist
models_dir = './models'
os.makedirs(models_dir, exist_ok=True)

# Save baseline model (using PyTorch standard method)
baseline_path = os.path.join(models_dir, 'baseline_resnet50.pth')
torch.save(baseline_model.state_dict(), baseline_path)
print(f"✅ Baseline ResNet50 saved to {baseline_path}")

# Save multi-stream models using their built-in save_model method
base_multi_channel_path = os.path.join(models_dir, 'base_multi_channel_large.pth')
base_multi_channel_large_model.save_model(base_multi_channel_path)
print(f"✅ BaseMultiChannelNetwork saved to {base_multi_channel_path}")

multi_channel_resnet_path = os.path.join(models_dir, 'multi_channel_resnet50.pth')
multi_channel_resnet50_model.save_model(multi_channel_resnet_path)
print(f"✅ MultiChannelResNetNetwork saved to {multi_channel_resnet_path}")

# Save training histories for future reference
history_dir = os.path.join(models_dir, 'history')
os.makedirs(history_dir, exist_ok=True)

# Convert torch tensors to lists for JSON serialization
def prepare_for_json(history):
    json_history = {}
    for key, value in history.items():
        if isinstance(value, list):
            # Check if list contains tensors
            if value and isinstance(value[0], torch.Tensor):
                json_history[key] = [float(v.item()) for v in value]
            else:
                # Convert all values to float for consistency
                json_history[key] = [float(v) if not isinstance(v, (list, tuple)) else v for v in value]
        else:
            json_history[key] = value
    return json_history

# Baseline history conversion
json_baseline_history = {}
for key, value in baseline_history.items():
    json_baseline_history[key] = [float(v) for v in value]
    
# Save histories
with open(os.path.join(history_dir, 'baseline_history.json'), 'w') as f:
    json.dump(json_baseline_history, f)

with open(os.path.join(history_dir, 'base_multi_channel_history.json'), 'w') as f:
    json.dump(prepare_for_json(base_multi_channel_large_history), f)
    
with open(os.path.join(history_dir, 'multi_channel_resnet_history.json'), 'w') as f:
    json.dump(prepare_for_json(multi_channel_resnet50_history), f)

print(f"✅ Training histories saved to {history_dir}")

# Save model metadata for easier reloading
metadata = {
    'baseline': {
        'model_type': 'CifarResNet50',
        'num_classes': 100,
        'parameters': baseline_params,
        'path': baseline_path
    },
    'base_multi_channel_large': {
        'model_type': 'BaseMultiChannelNetwork',
        'color_input_size': rgb_input_size,
        'brightness_input_size': brightness_input_size,
        'num_classes': 100,
        'parameters': large_dense_params,
        'path': base_multi_channel_path
    },
    'multi_channel_resnet50': {
        'model_type': 'MultiChannelResNetNetwork',
        'color_input_channels': input_channels_rgb,
        'brightness_input_channels': input_channels_brightness,
        'num_classes': 100,
        'parameters': resnet50_params,
        'path': multi_channel_resnet_path
    },
    'dataset': 'CIFAR-100',
    'training_date': time.strftime('%Y-%m-%d')
}

with open(os.path.join(models_dir, 'model_metadata.json'), 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"✅ Model metadata saved to {os.path.join(models_dir, 'model_metadata.json')}")
print("\n💾 All models and training data saved successfully!")

## 18. Summary

Summarize the results and findings from our multi-stream neural network experiments.

In [ ]:
# Summary
print("📋 Multi-Stream Neural Networks CIFAR-100 Training Summary")

# Training summary
print("\n🏋️‍♀️ Training Results:")
print(f"   Baseline ResNet50 final validation accuracy: {baseline_history['val_acc'][-1]:.2f}%")
print(f"   BaseMultiChannelNetwork final validation accuracy: {base_multi_channel_large_history['val_acc'][-1]:.2f}%")
print(f"   MultiChannelResNetNetwork final validation accuracy: {multi_channel_resnet50_history['val_acc'][-1]:.2f}%")

# Testing summary
print("\n🧪 Testing Results:")
print(f"   Baseline ResNet50 test accuracy: {baseline_results['test_acc']:.2f}%")
print(f"   BaseMultiChannelNetwork test accuracy: {base_multi_channel_large_results['test_acc']:.2f}%")
print(f"   MultiChannelResNetNetwork test accuracy: {multi_channel_resnet50_results['test_acc']:.2f}%")

# Pathway analysis summary
print("\n🔍 Pathway Analysis Summary:")
print("   BaseMultiChannelNetwork:")
print(f"      Combined accuracy: {base_multi_channel_large_pathway_analysis['combined_acc']:.2f}%")
print(f"      RGB pathway: {base_multi_channel_large_pathway_analysis['rgb_acc']:.2f}%, Brightness pathway: {base_multi_channel_large_pathway_analysis['brightness_acc']:.2f}%")

print("   MultiChannelResNetNetwork:")
print(f"      Combined accuracy: {multi_channel_resnet50_pathway_analysis['combined_acc']:.2f}%")
print(f"      RGB pathway: {multi_channel_resnet50_pathway_analysis['rgb_acc']:.2f}%, Brightness pathway: {multi_channel_resnet50_pathway_analysis['brightness_acc']:.2f}%")

# Create a summary table
summary_data = {
    'Model': ['Baseline ResNet50', 'BaseMultiChannelNetwork', 'MultiChannelResNetNetwork'],
    'Test Acc (%)': [
        f"{baseline_results['test_acc']:.2f}",
        f"{base_multi_channel_large_results['test_acc']:.2f}",
        f"{multi_channel_resnet50_results['test_acc']:.2f}"
    ],
    'RGB Pathway (%)': [
        'N/A',
        f"{base_multi_channel_large_pathway_analysis['rgb_acc']:.2f}",
        f"{multi_channel_resnet50_pathway_analysis['rgb_acc']:.2f}"
    ],
    'Brightness Pathway (%)': [
        'N/A',
        f"{base_multi_channel_large_pathway_analysis['brightness_acc']:.2f}",
        f"{multi_channel_resnet50_pathway_analysis['brightness_acc']:.2f}"
    ],
    'Parameters': [
        f"{sum(p.numel() for p in baseline_model.parameters()):,}",
        f"{large_dense_params:,}",
        f"{resnet50_params:,}"
    ]
}

# Use pandas to create a nice table
summary_df = pd.DataFrame(summary_data)
display(summary_df)

print("\n📝 Key Findings:")
print("   1. Multi-stream models can leverage both RGB and brightness information")
print("   2. The RGB pathway typically contributes more to accuracy than brightness")
print("   3. The combined model performs better than individual pathways")
print("   4. MultiChannelResNetNetwork architecture is more powerful but requires more parameters")

print("\n🎯 Next Steps:")
print("   1. Try different fusion strategies")
print("   2. Experiment with balancing pathway contributions")
print("   3. Apply to more complex datasets")
print("   4. Optimize model architectures based on pathway analysis")

print("\n✨ Thank you for exploring Multi-Stream Neural Networks! ✨")

# Visualize Model Predictions and Pathway Analysis
print("🔍 Visualizing model predictions and pathway analysis...")

# Get a batch of test data
test_batch = next(iter(test_loader))
# Handle different data formats - support both tuple and list types
if isinstance(test_batch, (list, tuple)) and len(test_batch) == 3:
    test_rgb, test_brightness, test_labels = test_batch
else:
    # Handle unexpected data format
    if not isinstance(test_batch, (list, tuple)):
        raise ValueError(f"Expected test_batch to be list or tuple, got {type(test_batch)}")
    elif len(test_batch) != 3:
        raise ValueError(f"Expected test_batch to have 3 elements, got {len(test_batch)}")
    else:
        raise ValueError("Unexpected data format from test_loader")

test_rgb = test_rgb.to(device)
test_brightness = test_brightness.to(device)
test_labels = test_labels.to(device)

# Get predictions from all models
with torch.no_grad():
    # Baseline model only takes RGB
    baseline_outputs = baseline_model(test_rgb)
    baseline_probs = torch.softmax(baseline_outputs, dim=1)
    baseline_preds = torch.argmax(baseline_probs, dim=1)
    
    # Multi-stream models take both inputs
    base_multi_channel_outputs = base_multi_channel_large_model(test_rgb, test_brightness)
    base_multi_channel_probs = torch.softmax(base_multi_channel_outputs, dim=1)
    base_multi_channel_preds = torch.argmax(base_multi_channel_probs, dim=1)
    
    multi_channel_resnet_outputs = multi_channel_resnet50_model(test_rgb, test_brightness)
    multi_channel_resnet_probs = torch.softmax(multi_channel_resnet_outputs, dim=1)
    multi_channel_resnet_preds = torch.argmax(multi_channel_resnet_probs, dim=1)
    
    # Get separate pathway predictions from multi-stream models using analyze_pathways
    base_color_outputs, base_brightness_outputs = base_multi_channel_large_model.analyze_pathways(test_rgb, test_brightness)
    base_color_probs = torch.softmax(base_color_outputs, dim=1)
    base_brightness_probs = torch.softmax(base_brightness_outputs, dim=1)
    base_color_preds = torch.argmax(base_color_probs, dim=1)
    base_brightness_preds = torch.argmax(base_brightness_probs, dim=1)
    
    resnet_color_outputs, resnet_brightness_outputs = multi_channel_resnet50_model.analyze_pathways(test_rgb, test_brightness)
    resnet_color_probs = torch.softmax(resnet_color_outputs, dim=1)
    resnet_brightness_probs = torch.softmax(resnet_brightness_outputs, dim=1)
    resnet_color_preds = torch.argmax(resnet_color_probs, dim=1)
    resnet_brightness_preds = torch.argmax(resnet_brightness_probs, dim=1)

# Move to CPU for visualization
test_rgb = test_rgb.cpu()
test_brightness = test_brightness.cpu()
test_labels = test_labels.cpu()
baseline_preds = baseline_preds.cpu()
base_multi_channel_preds = base_multi_channel_preds.cpu()
multi_channel_resnet_preds = multi_channel_resnet_preds.cpu()
base_color_preds = base_color_preds.cpu()
base_brightness_preds = base_brightness_preds.cpu()
resnet_color_preds = resnet_color_preds.cpu()
resnet_brightness_preds = resnet_brightness_preds.cpu()

# Select a subset of samples for visualization
n_samples = min(8, test_rgb.size(0))
sample_indices = np.random.choice(test_rgb.size(0), n_samples, replace=False)

# Plot original images, model predictions, and pathway contributions
fig, axes = plt.subplots(n_samples, 6, figsize=(18, 3*n_samples))
if n_samples == 1:
    axes = axes.reshape(1, -1)

for i, idx in enumerate(sample_indices):
    # Get RGB image
    rgb_img = test_rgb[idx].permute(1, 2, 0).numpy()
    rgb_img = np.clip(rgb_img, 0, 1)
    
    # Get brightness image
    brightness_img = test_brightness[idx, 0].numpy()
    
    # True label and predictions
    true_label = test_labels[idx].item()
    baseline_pred = baseline_preds[idx].item()
    multi_channel_pred = multi_channel_resnet_preds[idx].item()
    color_pathway_pred = resnet_color_preds[idx].item()
    brightness_pathway_pred = resnet_brightness_preds[idx].item()
    
    # Plot original RGB image
    axes[i, 0].imshow(rgb_img)
    axes[i, 0].set_title(f"True: {CIFAR100_FINE_LABELS[true_label]}")
    axes[i, 0].axis('off')
    
    # Plot brightness image
    axes[i, 1].imshow(brightness_img, cmap='gray')
    axes[i, 1].set_title("Brightness Channel")
    axes[i, 1].axis('off')
    
    # Plot baseline model prediction
    axes[i, 2].imshow(rgb_img)
    correct = true_label == baseline_pred
    title_color = 'green' if correct else 'red'
    axes[i, 2].set_title(f"Baseline: {CIFAR100_FINE_LABELS[baseline_pred]}", color=title_color)
    axes[i, 2].axis('off')
    
    # Plot multi-stream model prediction
    axes[i, 3].imshow(rgb_img)
    correct = true_label == multi_channel_pred
    title_color = 'green' if correct else 'red'
    axes[i, 3].set_title(f"Multi-Stream: {CIFAR100_FINE_LABELS[multi_channel_pred]}", color=title_color)
    axes[i, 3].axis('off')
    
    # Plot color pathway prediction
    axes[i, 4].imshow(rgb_img)
    correct = true_label == color_pathway_pred
    title_color = 'green' if correct else 'red'
    axes[i, 4].set_title(f"Color Path: {CIFAR100_FINE_LABELS[color_pathway_pred]}", color=title_color)
    axes[i, 4].axis('off')
    
    # Plot brightness pathway prediction
    axes[i, 5].imshow(brightness_img, cmap='gray')
    correct = true_label == brightness_pathway_pred
    title_color = 'green' if correct else 'red'
    axes[i, 5].set_title(f"Brightness Path: {CIFAR100_FINE_LABELS[brightness_pathway_pred]}", color=title_color)
    axes[i, 5].axis('off')

plt.tight_layout()
plt.show()

# Visualize pathway feature importance
plt.figure(figsize=(12, 6))

# Get pathway importance
base_importance = base_multi_channel_large_model.get_pathway_importance()
resnet_importance = multi_channel_resnet50_model.get_pathway_importance()

# Plot importance values
labels = ['Color Pathway', 'Brightness Pathway']
base_values = [base_importance['color_pathway'], base_importance['brightness_pathway']]
resnet_values = [resnet_importance['color_pathway'], resnet_importance['brightness_pathway']]

x = np.arange(len(labels))
width = 0.35

plt.bar(x - width/2, base_values, width, label='BaseMultiChannelNetwork')
plt.bar(x + width/2, resnet_values, width, label='MultiChannelResNetNetwork')

plt.ylabel('Relative Importance')
plt.title('Pathway Importance Analysis')
plt.xticks(x, labels)
plt.ylim(0, 1)
plt.legend()
plt.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Visualization complete!")

## Summary and Conclusions

In this notebook, we've built, trained, and evaluated multi-stream neural networks for CIFAR-100 classification:

1. **Data Preparation**
   - Processed CIFAR-100 data into RGB and brightness streams
   - Applied unified data augmentation with the same transformations for all models
   - Created robust data processing pipelines with batch processing and memory optimization

2. **Baseline Model**
   - Trained a non-pretrained ResNet50 baseline modified for CIFAR-100
   - Used proper regularization (weight decay, early stopping)
   - Applied learning rate scheduling for optimal convergence

3. **Multi-Stream Models**
   - Implemented two multi-stream architectures:
     - BaseMultiChannelNetwork (dense/fully-connected)
     - MultiChannelResNetNetwork (CNN with residual connections)
   - Both models process RGB and brightness data in parallel pathways
   - Used shared classifier for final prediction
   - Leveraged the models' built-in APIs for efficient training and evaluation:
     - `compile()`: Set up optimizer, loss function, and metrics
     - `fit_dataloader()`: Trained models directly with DataLoaders for on-the-fly augmentation
     - `fit()`: Alternative training method using NumPy arrays/tensors with built-in early stopping and scheduling
     - `evaluate()`: Evaluated models on test data
     - `analyze_pathways()`: Analyzed pathway contributions and importance

4. **Fair Comparison Framework**
   - All models trained with:
     - Same optimization settings (Adam/AdamW optimizer)
     - Same regularization (weight decay, optional dropout)
     - Same learning rate scheduling
     - Same early stopping patience
     - Same data augmentation pipeline

5. **Results and Analysis**
   - Compared performance on test set
   - Identified classes where multi-stream processing provides advantages
   - Analyzed model convergence and learning patterns
   - Used pathway analysis to understand feature importance in each stream

The results demonstrate that separating color and brightness information into distinct processing pathways can lead to improved classification performance compared to a standard RGB-only approach.

### Key Advantage of the Enhanced Training Pipeline

By implementing the `fit_dataloader()` method in our multi-stream models, we achieved:
- **True on-the-fly augmentation** - Each batch receives freshly augmented data
- **Consistent training approach** across all models (baseline and multi-stream)
- **Better generalization** through exposure to more diverse augmented samples
- **Memory efficiency** by not storing pre-extracted augmented data
- **Simplified workflow** - direct DataLoader usage without manual data extraction

### Proper API Usage

For Multi-Stream Neural Networks, the recommended API pattern is:
1. Create model with `model = create_model(...)` or direct constructor
2. Compile model with `model.compile(...)`
3. Train model with `model.fit_dataloader(train_loader, val_loader, ...)` for on-the-fly augmentation
4. Or train with `model.fit(...)` using NumPy arrays or tensors if preferred
5. Evaluate with `model.evaluate(...)`
6. Analyze pathways with `model.analyze_pathways(...)`

This provides a clean, flexible interface that is consistent across all model architectures.

### Next Steps
- Perform additional ablation studies on different pathway configurations
- Explore other fusion mechanisms beyond shared classifiers
- Apply these models to more complex datasets and real-world applications
- Analyze computational efficiency and potential optimizations